In [12]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, save_model, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, Input
from tensorflow.keras.regularizers import l2
from tensorflow.keras.applications import VGG16
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

In [13]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

58889256/58889256 [==============================] - 6s 0us/step


In [14]:
# Freeze the base model
for layer in base_model.layers:
    layer.trainable = False

In [15]:
# Add custom layers on top of the base model
x = base_model.output
x = Flatten()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(6, activation='softmax')(x)  # Adjust the number of units based on the number of classes

In [16]:
model = Model(inputs=base_model.input, outputs=x)

In [17]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [18]:
for layer in base_model.layers[-3:]:
    layer.trainable = True

In [19]:
# Compile the model with a low learning rate
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])

In [36]:
# Create the Sequential model
model = Sequential([
    Input(shape=(224, 224, 3)),  # Define the input shape with the Input layer
    
    Conv2D(32, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(0.25),

    Conv2D(64, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(0.25),

    Conv2D(128, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(0.25),

    Flatten(),

    Dense(128, activation='relu', kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    Dropout(0.5),

    Dense(64, activation='relu', kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    Dropout(0.5),

    Dense(32, activation='relu', kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    Dropout(0.5),

    Dense(9, activation='softmax')
])

In [37]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [38]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 222, 222, 32)      896       
                                                                 
 batch_normalization_6 (Bat  (None, 222, 222, 32)      128       
 chNormalization)                                                
                                                                 
 max_pooling2d_6 (MaxPoolin  (None, 111, 111, 32)      0         
 g2D)                                                            
                                                                 
 dropout_8 (Dropout)         (None, 111, 111, 32)      0         
                                                                 
 conv2d_7 (Conv2D)           (None, 109, 109, 64)      18496     
                                                                 
 batch_normalization_7 (Bat  (None, 109, 109, 64)     

In [39]:
# Data Augmentation and Generators
train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=20, width_shift_range=0.2,
                                   height_shift_range=0.2, shear_range=0.2, zoom_range=0.2, 
                                   horizontal_flip=True, fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(r"C:\Users\karti\Desktop\Skin cancer ISIC The International Skin Imaging Collaboration\Train", target_size=(224, 224), 
                                                    batch_size=64, class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(r"C:\Users\karti\Desktop\Skin cancer ISIC The International Skin Imaging Collaboration\Test", target_size=(224, 224), 
                                                        batch_size=64, class_mode='categorical')

Found 2239 images belonging to 9 classes.
Found 118 images belonging to 9 classes.


In [40]:
data_dir = r"C:\Users\karti\Desktop\Skin cancer ISIC The International Skin Imaging Collaboration\Train"

print("Class indices:", train_generator.class_indices)
for cls in train_generator.class_indices:
    cls_dir = os.path.join(data_dir, cls)
    num_images = len(os.listdir(cls_dir))
    print(f"Class {cls} has {num_images} images")

Class indices: {'actinic keratosis': 0, 'basal cell carcinoma': 1, 'dermatofibroma': 2, 'melanoma': 3, 'nevus': 4, 'pigmented benign keratosis': 5, 'seborrheic keratosis': 6, 'squamous cell carcinoma': 7, 'vascular lesion': 8}
Class actinic keratosis has 114 images
Class basal cell carcinoma has 376 images
Class dermatofibroma has 95 images
Class melanoma has 438 images
Class nevus has 357 images
Class pigmented benign keratosis has 462 images
Class seborrheic keratosis has 77 images
Class squamous cell carcinoma has 181 images
Class vascular lesion has 139 images


In [41]:
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_generator.classes),
    y=train_generator.classes
)

class_weights_dict = dict(enumerate(class_weights))
print("Class weights:", class_weights_dict)

Class weights: {0: 2.182261208576998, 1: 0.6616430260047281, 2: 2.6187134502923977, 3: 0.5679857940131913, 4: 0.6968565203859322, 5: 0.5384800384800384, 6: 3.230880230880231, 7: 1.3744628606507059, 8: 1.7897681854516387}


In [42]:
early_stopping = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True,verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-6,verbose=1)

In [43]:
# Train the Model
history = model.fit(train_generator, epochs=50, validation_data=validation_generator, callbacks=[reduce_lr,early_stopping],class_weight = class_weights_dict)

Epoch 1/50
35/35 [==============================] - 145s 4s/step - loss: 3.5254 - accuracy: 0.1242 - val_loss: 4.6832 - val_accuracy: 0.1356 - lr: 0.0010
Epoch 2/50
35/35 [==============================] - 134s 4s/step - loss: 3.3406 - accuracy: 0.1577 - val_loss: 3.3613 - val_accuracy: 0.1356 - lr: 0.0010
Epoch 3/50
35/35 [==============================] - 134s 4s/step - loss: 3.2429 - accuracy: 0.1724 - val_loss: 3.3839 - val_accuracy: 0.1356 - lr: 0.0010
Epoch 4/50
35/35 [==============================] - 132s 4s/step - loss: 3.1038 - accuracy: 0.1818 - val_loss: 3.1611 - val_accuracy: 0.1017 - lr: 0.0010
Epoch 5/50
35/35 [==============================] - 133s 4s/step - loss: 2.9300 - accuracy: 0.2095 - val_loss: 3.0602 - val_accuracy: 0.1271 - lr: 0.0010
Epoch 6/50
35/35 [==============================] - 132s 4s/step - loss: 2.8112 - accuracy: 0.2318 - val_loss: 2.9572 - val_accuracy: 0.1949 - lr: 0.0010
Epoch 7/50
35/35 [==============================] - 134s 4s/step - loss: 2.7

In [46]:
# Save the Trained Model
model.save("trained_model.keras")

# Save the model in TensorFlow SavedModel format
model.save('model.h5')

# Load the model from TensorFlow SavedModel format
model = tf.keras.models.load_model('model.h5')

model = tf.keras.models.load_model('model.h5')

In [48]:
# Evaluate the Model on Test Data
test_generator = test_datagen.flow_from_directory(r"C:\Users\karti\Desktop\Skin cancer ISIC The International Skin Imaging Collaboration\Test", target_size=(224, 224), 
                                                  batch_size=64, class_mode='categorical', shuffle=True)

test_loss, test_accuracy = model.evaluate(test_generator)
print(f'Test accuracy: {test_accuracy * 100:.2f}%')

Found 118 images belonging to 9 classes.
2/2 [==============================] - 6s 3s/step - loss: 1.9839 - accuracy: 0.4068
Test accuracy: 40.68%


In [49]:
# Get the ground truth labels
test_labels = test_generator.classes
print(test_labels)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 6 6 6 7 7 7 7 7 7 7 7 7 7 7 7
 7 7 7 7 8 8 8]


In [50]:
# Predict the classes
predictions = model.predict(test_generator)
predicted_classes = np.argmax(predictions, axis=1)

2/2 [==============================] - 5s 1s/step


In [51]:
# Predict the classes
predictions = model.predict(test_generator)
predicted_classes = np.argmax(predictions, axis=1)
# Print classification report
print(classification_report(test_labels, predicted_classes, target_names=test_generator.class_indices.keys()))

# Print confusion matrix
print(confusion_matrix(test_labels, predicted_classes))

2/2 [==============================] - 5s 2s/step
                            precision    recall  f1-score   support

         actinic keratosis       0.08      0.12      0.10        16
      basal cell carcinoma       0.12      0.06      0.08        16
            dermatofibroma       0.07      0.06      0.07        16
                  melanoma       0.00      0.00      0.00        16
                     nevus       0.19      0.38      0.25        16
pigmented benign keratosis       0.09      0.12      0.11        16
      seborrheic keratosis       0.00      0.00      0.00         3
   squamous cell carcinoma       0.33      0.12      0.18        16
           vascular lesion       0.00      0.00      0.00         3

                  accuracy                           0.12       118
                 macro avg       0.10      0.10      0.09       118
              weighted avg       0.12      0.12      0.11       118

[[2 1 2 0 5 4 0 1 1]
 [3 1 1 0 3 4 2 0 2]
 [5 2 1 0 2 3 1 1 1]
